# TensorFlow/Kerasによる分類(学習)


In [ ]:
# ライブラリのインポート
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

from keras import backend as K
from keras.preprocessing.image import load_img
import matplotlib.pyplot as plt
import numpy as np
import os
import re
import h5py

In [ ]:
# 各種パラメータ
img_rows, img_cols = 64, 64
batch_size = 128
num_classes = 3
epochs = 60

In [ ]:
# 学習データの読み込み
image_data_set = np.empty((0,img_cols,img_rows,3), dtype=np.uint8)
label_data_set = np.empty((0,1), dtype=np.uint8)

label = 0
for k in ['yukiho', 'miki', 'yayoi']:
    parent_path = "data/train/%s" % (k)
    for file in os.listdir(parent_path):
        if re.search(r".jpg", file):
            file_full = parent_path + '/' + file
            im_reading = np.array( load_img(file_full).resize((img_cols,img_rows)))
            image_data_set = np.append(image_data_set, [im_reading], axis=0)
            label_data_set = np.append(label_data_set, [[label]], axis=0)
    label += 1

In [ ]:
# 学習データと検証データに分割
from sklearn.cross_validation import train_test_split
x_train, x_test, y_train, y_test = train_test_split(image_data_set, label_data_set, test_size=0.10, random_state=10)


In [ ]:
# 形式変換 (TheanoとTensorFlowでデータの保持順が異なるため)
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 3, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 3, img_rows, img_cols)
    input_shape = (3, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 3)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 3)
    input_shape = (img_rows, img_cols, 3)


In [ ]:
# モデルの定義
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

print(model.summary())

In [ ]:
# 学習の実施 （注： この処理は時間がかかります）
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))


In [ ]:
# 検証
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
# モデルの保存
import h5py
model.save("data/imas_yukiho_miki_yayoi.h5")
